In [1]:
!pip install deepchem
!pip install pysmiles
!pip install openpyxl
# !pip install rdkit

# !pip install MolGraphConvFeaturizer
!pip install PubChemPy
!pip install PyDrive
!pip install xlrd==2.0.1
!pip install Pandas==1.3.5
# !pip install Pandas==1.1.5
!pip install  gdown
import deepchem as dc
from rdkit import Chem
print("hjjbjh")
from pysmiles import read_smiles
import networkx as nx
from deepchem.feat import MolGraphConvFeaturizer
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
#   gauth = GoogleAuth()
#   gauth.credentials = GoogleCredentials.get_application_default()
#   drivea = GoogleDrive(gauth)
#   drive.mount('/content/drive')
def normalize1(X, means1=None, std1=None, means2=None, std2=None, feat_filt=None, norm='tanh_norm'):
    if std1 is None:
        std1 = np.nanstd(X, axis=0)
    if feat_filt is None:
        feat_filt = std1!=0
    X = X[:,feat_filt]
    X = np.ascontiguousarray(X)
    if means1 is None:
        means1 = np.mean(X, axis=0)
    X = (X-means1)/std1[feat_filt]
    if norm == 'norm':
        return(X, means1, std1, feat_filt)
    elif norm == 'tanh':
        return(np.tanh(X), means1, std1, feat_filt)
    elif norm == 'tanh_norm':
        X = np.tanh(X)
        if means2 is None:
            means2 = np.mean(X, axis=0)
        if std2 is None:
            std2 = np.std(X, axis=0)
        X = (X-means2)/std2
        X[:,std2==0]=0
        return(X, means1, std1, means2, std2, feat_filt) 
    
    
def get_data():
  !gdown https://drive.google.com/uc?id=1m69fQ5l7q3dwQeud5M1nUVNhmrTGFbZc
  smiles=pd.read_excel('pubchem.xls', header=None)
  smiles=np.array(smiles)

#   !gdown https://drive.google.com/uc?id=15bNKK7tacCJIFzvt5y4WfU6uKbPdYtA6
  !gdown --id 1z1sWG4E9BqipP0iczLrbR4qxSa4KG95_
  data_to_repeat=pd.read_excel('pcbi.1006752.s004.xls', header=None)
  data_to_repeat=np.array(data_to_repeat)
  
#   !gdown --id 1TThHsLyORlcHuEBgOad20SQUYH88O9mm
#   data_to_repeat=pd.read_excel('labels1.xlsx', header=None)
#   data_to_repeat=np.array(data_to_repeat)
    
  !gdown https://drive.google.com/uc?id=1m69fQ5l7q3dwQeud5M1nUVNhmrTGFbZc
  unique_drugs=pd.read_excel('pubchem.xls', header=None)
  unique_drugs=np.array(unique_drugs)         

#   !gdown https://drive.google.com/uc?id=1IMr5zMLRAXC5iE2MAHJbKmHf2-0DCZNd
# #   !gdown --id 1bBJUFBA4Tm9YdE5OxA1mbUfBI8B7wqcr
#   feature_cell=pd.read_excel('final_feature_cell.xlsx', header=None)
#   feature_cell=np.array(feature_cell)

  !gdown --id 1NoUKCRZ0CkNVFP1-578x_L0otPUMkz6T
  feature_cell=pd.read_excel('unique934_cell.xlsx',header=None)
  feature_cell=np.array(feature_cell)
  a=np.zeros((1,934))
  feature_cell[22,1:]=a
  feature_cell[36,1:]=a
#   !gdown --id 109nyFVOO_P9DdyrhWzbNg0FBB_Y8gD58
#   feature_cell=pd.read_excel('final_deep_cell.xlsx',header=None)
#   feature_cell=np.array(feature_cell)

#   !gdown https://drive.google.com/uc?id=1YTe0v5PzwjlgPqo3OTU4FMghOFAHgeeC
  !gdown --id 1lLKI0xPEbTBlkpu_6F3yoSYqf7pazg8d
  deleted_index=pd.read_excel('deleted_index.xls', header=None)
  deleted_index=np.array(deleted_index)  

  return smiles[1:,1],data_to_repeat[1:,:],unique_drugs[1:,0],feature_cell,deleted_index


def atom_features(atom):
    return np.array(one_of_k_encoding_unk(atom.GetSymbol(),
                                          ['C', 'N', 'O', 'S', 'F', 'Si', 'P', 'Cl', 'Br', 'Mg', 'Na', 'Ca', 'Fe', 'As',
                                           'Al', 'I', 'B', 'V', 'K', 'Tl', 'Yb', 'Sb', 'Sn', 'Ag', 'Pd', 'Co', 'Se',
                                           'Ti', 'Zn', 'H', 'Li', 'Ge', 'Cu', 'Au', 'Ni', 'Cd', 'In', 'Mn', 'Zr', 'Cr',
                                           'Pt', 'Hg', 'Pb', 'Unknown']) +
                    one_of_k_encoding(atom.GetDegree(), [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]) +
                    one_of_k_encoding_unk(atom.GetTotalNumHs(), [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]) +
                    one_of_k_encoding_unk(atom.GetImplicitValence(), [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]) +
                    [atom.GetIsAromatic()])


def one_of_k_encoding(x, allowable_set):
    if x not in allowable_set:
        raise Exception("input {0} not in allowable set{1}:".format(x, allowable_set))
    return list(map(lambda s: x == s, allowable_set))


def one_of_k_encoding_unk(x, allowable_set):
    """Maps inputs not in the allowable set to the last element."""
    if x not in allowable_set:
        x = allowable_set[-1]
    return list(map(lambda s: x == s, allowable_set))


def smile_to_graph(smile):
    mol = Chem.MolFromSmiles(smile)

    c_size = mol.GetNumAtoms()

    features = []
    for atom in mol.GetAtoms():
        feature = atom_features(atom)
        features.append(feature / sum(feature))

    edges = []
    for bond in mol.GetBonds():
        edges.append([bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()])
    g = nx.Graph(edges).to_directed()
    edge_index = []
    for e1, e2 in g.edges:
        edge_index.append([e1, e2])

    return c_size, features, edge_index


def graph_node(smiles):
  node=[]
  adj=[]
  max=0
  final_feature=[]
  final_adjacency=[]
  for i  in range(len(smiles)):
     m=smiles[i]
    #  aa=m#.item((0))
     featurizer = MolGraphConvFeaturizer(use_edges=True,use_chirality=True)
     out = featurizer.featurize(m)
     feature=out[0].node_features
        
#      featurizer = DMPNNFeaturizer()
#      out = featurizer.featurize(m)
#      feature=out[0].node_features
    
     ma=feature.shape[0]
     n=feature.shape[1]
     if(ma>max):
       max=ma
    # out[0].edge_featuwres.shape
     molecules=read_smiles(m)
     adjacency1=nx.to_numpy_array(molecules)
     node.append(feature)
     adj.append(adjacency1)
#   f_n=np.zeros((max,n))
#   f_a=np.zeros((max,max))
  feature=[]
  adjacency=[]
  for i  in range(len(smiles)):
    n1=node[i]
    a=adj[i]
    f_n=np.zeros((max,n))
    f_a=np.zeros((max,max))
    f_n[0:n1.shape[0],]=n1
    f_a[0:n1.shape[0],0:n1.shape[0]]=a
    final_feature.append(f_n)
    final_adjacency.append(f_a)


  return final_feature,final_adjacency 


def graph_node_edge(smiles):
  node=[]
  adj=[]
  max=0
  max1=0
  final_feature=[]
  final_adjacency=[]
  for i  in range(len(smiles)):
     m=smiles[i]
    #  aa=m#.item((0))
    #  featurizer = MolGraphConvFeaturizer(use_edges=True,use_chirality=True)
    #  out = featurizer.featurize(m)
    #  feature=out[0].node_features
     num_atom,feature,edge_index=smile_to_graph(m)  
#      featurizer = DMPNNFeaturizer()
#      out = featurizer.featurize(m)
#      feature=out[0].node_features
     feature=np.stack( feature, axis=0)
     edge_index=np.stack( edge_index, axis=0)
    
     ma=num_atom
     n=feature.shape[1]
     if(ma>max):
       max=ma
    # out[0].edge_featuwres.shape
     node.append(feature)

     
     ne=edge_index.shape[0]
     if(ne>max1):
       max1=ne
    # out[0].edge_featuwres.shape
     adj.append(edge_index)
#   f_n=np.zeros((max,n))
#   f_a=np.zeros((max,max))
  feature=[]
  adjacency=[]
  for i  in range(len(smiles)):
    n1=node[i]
    a=adj[i]
    f_n=np.zeros((max,n))
    f_n[0:n1.shape[0],]=n1
    f_a=np.zeros((max1,2))
    f_a[0:a.shape[0],]=a
    final_feature.append(f_n)
    final_adjacency.append(f_a)

  return final_feature,final_adjacency 


def repeat_smiles(data_to_repeat,unique_drugs,feature,adjacency,deleted_index,synergy,senstivity):
  d1=data_to_repeat[:,0]
  d2=data_to_repeat[:,1]
  f_drug1=[]
  f_drug2=[]
  a_drug1=[]
  a_drug2=[]
  synergy1=[]
  senstivity1=[]
  for i in range(len(d1)):
    n1=d1[i]
    n2=d2[i]
    k1= [m for m, v in enumerate(unique_drugs) if n1 in v]
    k2=[m for m, v in enumerate(unique_drugs) if n2 in v]
    if(i not in deleted_index):
      f_drug1.append(feature[k1[0]])
      f_drug2.append(feature[k2[0]])
      a_drug1.append(adjacency[k1[0]])
      a_drug2.append(adjacency[k2[0]])
      synergy1.append(synergy[i,])
      senstivity1.append(senstivity[i,])

  return f_drug1,f_drug2,a_drug1,a_drug2,synergy1,senstivity1

def repeat_smiles1(data_to_repeat,unique_drugs,feature,adjacency,unique_cell):
  unique_feature=unique_cell[:,1:]
  unique_name=unique_cell[:,0]
  d1=data_to_repeat[:,0]
  d2=data_to_repeat[:,1]
  c=data_to_repeat[:,2]
  f_drug1=[]
  f_drug2=[]
  a_drug1=[]
  a_drug2=[]
  feature_cell=[]
  for i in range(len(d1)):
    n1=d1[i]
    n2=d2[i]
    cc=c[i]
    k1= [m for m, v in enumerate(unique_drugs) if n1 in v]
    k2=[m for m, v in enumerate(unique_drugs) if n2 in v]
    cc1=[m for m, v in enumerate(unique_name) if cc in v]
    f_drug1.append(feature[k1[0]])
    f_drug2.append(feature[k2[0]])
    a_drug1.append(adjacency[k1[0]])
    a_drug2.append(adjacency[k2[0]])
    feature_cell.append(unique_feature[cc1[0]])

  return f_drug1,f_drug2,a_drug1,a_drug2,feature_cell 

def repeat_smiles_deep(data_to_repeat,unique_drugs,feature,adjacency,unique_cell):
  unique_feature=unique_cell[:,1:]
  unique_name=unique_cell[:,0]
  d1=data_to_repeat[:,0]
  d2=data_to_repeat[:,1]
  c=data_to_repeat[:,2]
  f_drug1=[]
  f_drug2=[]
  a_drug1=[]
  a_drug2=[]
  feature_cell=[]
  f=np.zeros((feature[0].shape[0],feature[0].shape[1]))
  a=np.zeros((adjacency[0].shape[0],adjacency[0].shape[1]))
  cell=np.zeros((unique_cell[0].shape[0]))
  for i in range(len(d1)):
    n1=d1[i]
    n2=d2[i]
    cc=c[i]
    k1= [m for m, v in enumerate(unique_drugs) if n1 in v]
    k2=[m for m, v in enumerate(unique_drugs) if n2 in v]
    cc1=[m for m, v in enumerate(unique_name) if cc in v]
    if(cc1):
        f_drug1.append(feature[k1[0]])
        f_drug2.append(feature[k2[0]])
        a_drug1.append(adjacency[k1[0]])
        a_drug2.append(adjacency[k2[0]])
        feature_cell.append(unique_feature[cc1[0]])
    else:
        f_drug1.append(f)
        f_drug2.append(f)
        a_drug1.append(a)
        a_drug2.append(a)
        feature_cell.append(cell)

  return f_drug1,f_drug2,a_drug1,a_drug2,feature_cell

def split_data(feature_cell,synergy,senstivity):
  row=feature_cell.shape[0]
  col=feature_cell.shape[1]
  input1=np.zeros((row,col+1))
  # a=np.array('i')
  for i in range(len(feature_cell)):
    input1[i,0:col]=feature_cell[i,:]
    input1[i,col]=i


  index_train=[]
  index_test=[]
  output = np.c_[synergy,senstivity ]
#   dataset1=np.c_[input11,output1]
#   dataset=np.random.shuffle(dataset1)
#   input1=dataset[:,0:len(dataset[0])-2]
#   output=dataset[:,-2:]
  x_train, x_test, y_train, y_test = train_test_split(input1, output,test_size=.20)
  train_synergy=y_train[:,0]
  train_senstivity=y_train[:,1]
  test_synergy=y_test[:,0]
  test_senstivity=y_test[:,1]
  for i in range(len(x_train)):#.shape[0]):
    for j in range(len(input1)):#.shape[0]):
      if(all(x_train[i]==input1[j])):
        index_train.append(j)
        

  for ii in range(len(x_test)):#.shape[0]):
    for jj in range(len(input1)):#.shape[0]):
      if(all(x_test[ii]==input1[jj])):
        index_test.append(jj)
        

  return  train_synergy, train_senstivity,test_synergy, test_senstivity,index_train,index_test


def split_data1(feature_cell,synergy,senstivity):
  row=feature_cell.shape[0]
  col=feature_cell.shape[1]
  input1=np.zeros((row,col+1))
  # a=np.array('i')
  k=0
  for i in range(len(feature_cell)):
    if((senstivity[i])):
        input1[k,0:col]=feature_cell[i,:]
        input1[k,col]=i
        k=k+1


  index_train=[]
  index_test=[]
  output=[]
  for i in range(len(feature_cell)):
    if((senstivity[i])):
      output.append (np.c_[synergy[i],senstivity[i] ])
#   dataset1=np.c_[input11,output1]
#   dataset=np.random.shuffle(dataset1)
#   input1=dataset[:,0:len(dataset[0])-2]
#   output=dataset[:,-2:]
  output= np.array(output)
  x_train, x_test, y_train, y_test = train_test_split(input1, output,test_size=.20)
  train_synergy=y_train[:,0]
  train_senstivity=y_train[:,1]
  test_synergy=y_test[:,0]
  test_senstivity=y_test[:,1]
  for i in range(len(x_train)):#.shape[0]):
    for j in range(len(input1)):#.shape[0]):
      if(all(x_train[i]==input1[j])):
        index_train.append(j)
        

  for ii in range(len(x_test)):#.shape[0]):
    for jj in range(len(input1)):#.shape[0]):
      if(all(x_test[ii]==input1[jj])):
        index_test.append(jj)
        

  return  index_train,index_test


def index(m):
    if m==1:
      !gdown --id 1b82ry7sSPqPIcJSr-i4rmC9XYp7Z1CpH
      test_ind=pd.read_excel('ind1_test.xls', header=None)
      test_ind=np.array(test_ind)
    
      !gdown --id 1aAaYK1EG5AVlWtwHlsajVJo4Qx-u2ig3
      train_ind=pd.read_excel('ind1_train.xls', header=None)
      train_ind=np.array(train_ind)

    
    if m==2:
        !gdown --id 1EXKurPZ5ScjiZM0OG-bSv7IPXyKKW6B5
        test_ind=pd.read_excel('ind2_test.xls', header=None)
        test_ind=np.array(test_ind) 
        
        !gdown --id 1-rayr-tZdwX4DDtSHChkoSdgEK1P3jWN
        train_ind=pd.read_excel('ind2_train.xls', header=None)
        train_ind=np.array(train_ind)

    if m==3:
        !gdown --id 19O4yKBUCAyPrvdUne2IhwOY73A4y7pzT
        test_ind=pd.read_excel('ind3_test.xls', header=None)
        test_ind=np.array(test_ind) 
        
        !gdown --id 1YNZBGdL--Ww9ZSGWUKmeC26DdJDywrR4
        train_ind=pd.read_excel('ind3_train.xls', header=None)
        train_ind=np.array(train_ind)

        
    if m==4:
        !gdown --id 1sJ3ksFbMOQoBTqNw5Ddv7bGgD9YESRGn
        test_ind=pd.read_excel('ind4_test.xls', header=None)
        test_ind=np.array(test_ind) 
        
        !gdown --id 1-P14BxLrpnYS-9TbsvyATKtFTFG7QsLO
        train_ind=pd.read_excel('ind4_train.xls', header=None)
        train_ind=np.array(train_ind)

        
    if m==5:
        !gdown --id 1iZo5wJgiUOGBAVRy0Hw6vXngFHe9ciFH
        test_ind=pd.read_excel('ind5_test.xls', header=None)
        test_ind=np.array(test_ind) 
        
        !gdown --id 1xlFp8g1gDGPa_Sf0zgrPWm3hnRCSkAxp
        train_ind=pd.read_excel('ind5_train.xls', header=None)
        train_ind=np.array(train_ind)

    return test_ind,train_ind


def index1(m):
    if m==1:
      !gdown --id 10a4cV1tOqomBo7RzWqnbs6j5-RGcBNjS
      test_ind=pd.read_excel('another_ind1_test.xls', header=None)
      test_ind=np.array(test_ind)
    
      !gdown --id 1Kn8Y2iGCH7DLwq72s_QmouA0Db1WBfVR
      train_ind=pd.read_excel('another_ind1_train.xls', header=None)
      train_ind=np.array(train_ind)

    
    if m==2:
        !gdown --id 1WR4kDyNO6Xo-P-z3O2q6M_tHr6Soj6NZ
        test_ind=pd.read_excel('another_ind2_test.xls', header=None)
        test_ind=np.array(test_ind) 
        
        !gdown --id 1aegtCzmVg9znXPTUvpzf2zLkmcjEeQ22
        train_ind=pd.read_excel('another_ind2_train.xls', header=None)
        train_ind=np.array(train_ind)

    if m==3:
        !gdown --id 1W29vAF6mOoKmacbibxzTNQCfp42VVYYN
        test_ind=pd.read_excel('another_ind3_test.xls', header=None)
        test_ind=np.array(test_ind) 
        
        !gdown --id 1B8h0ewSo4XJCvSWBdS5oiQiwtAL3H72T
        train_ind=pd.read_excel('another_ind3_train.xls', header=None)
        train_ind=np.array(train_ind)

        
    if m==4:
        !gdown --id 1tUXsGGT_-JdJTpT5sYEBB3zNFnh60tJR
        test_ind=pd.read_excel('another_ind4_test.xls', header=None)
        test_ind=np.array(test_ind) 
        
        !gdown --id 1T9H0O6bzr2d01cPnSyE-vYk_9doKKBHZ
        train_ind=pd.read_excel('another_ind4_train.xls', header=None)
        train_ind=np.array(train_ind)

        
    if m==5:
        !gdown --id 1p1kOBJQRhsavnOwgeb_3z-gBqLSS08sE
        test_ind=pd.read_excel('another_ind5_test.xls', header=None)
        test_ind=np.array(test_ind) 
        
        !gdown --id 1hhw-ITuxUnKwTlLveS1MJER_KM580SXa
        train_ind=pd.read_excel('another_ind5_train.xls', header=None)
        train_ind=np.array(train_ind)

    return test_ind,train_ind


def train_test_input(f_drug1,a_drug1,f_drug2,a_drug2,cell_line,index_train,index_test,synery,senstivity):
  train_f_drug1=[]
  train_a_drug1=[]
  train_f_drug2=[]
  train_a_drug2=[]
  train_cell_line=[]
  train_synergy=[]
  train_senstivity=[]
  test_f_drug1=[]
  test_a_drug1=[]
  test_f_drug2=[]
  test_a_drug2=[]
  test_cell_line=[]
  test_synergy=[]
  test_senstivity=[]
  for i in range(len(index_train)):
      train_a_drug1.append(a_drug1[index_train[i]])
      train_a_drug2.append(a_drug2[index_train[i]])
      train_f_drug1.append(f_drug1[index_train[i]])
      train_f_drug2.append(f_drug2[index_train[i]])
      train_cell_line.append(cell_line[index_train[i]])
      train_synergy.append(synergy[index_train[i]])
      train_senstivity.append(senstivity[index_train[i]])
  for ii in range(len(index_test)):
      test_a_drug1.append(a_drug1[index_test[ii]])
      test_a_drug2.append(a_drug2[index_test[ii]])
      test_f_drug1.append(f_drug1[index_test[ii]])
      test_f_drug2.append(f_drug2[index_test[ii]])
      test_cell_line.append(cell_line[index_test[ii]])
      test_synergy.append(synergy[index_test[ii]])
      test_senstivity.append(senstivity[index_test[ii]])

  return train_f_drug1,train_a_drug1,train_f_drug2,train_a_drug2,train_cell_line,test_f_drug1,test_a_drug1,test_f_drug2,test_a_drug2,test_cell_line,train_synergy,train_senstivity,test_synergy,test_senstivity

def train_test_input1(f_drug1,a_drug1,f_drug2,a_drug2,cell_line,index_test,synergy,senstivity):
  train_f_drug1=[]
  train_a_drug1=[]
  train_f_drug2=[]
  train_a_drug2=[]
  train_cell_line=[]
  train_synergy=[]
  train_senstivity=[]
  test_f_drug1=[]
  test_a_drug1=[]
  test_f_drug2=[]
  test_a_drug2=[]
  test_cell_line=[]
  test_synergy=[]
  test_senstivity=[]
    
  for i in range(len(f_drug1)):
     ind=i+1
     if(ind not in index_test):
          train_a_drug1.append(a_drug1[i])
          train_a_drug2.append(a_drug2[i])
          train_f_drug1.append(f_drug1[i])
          train_f_drug2.append(f_drug2[i])
          train_cell_line.append(cell_line[i])
          train_synergy.append(synergy[i])
          train_senstivity.append(senstivity[i])
  for ii in range(len(index_test)):
#       ind1=ii-1
      x=index_test[ii]-1
      n=x[0]
      test_a_drug1.append(a_drug1[n])
      test_a_drug2.append(a_drug2[n])
      test_f_drug1.append(f_drug1[n])
      test_f_drug2.append(f_drug2[n])
      test_cell_line.append(cell_line[n])
      test_synergy.append(synergy[n])
      test_senstivity.append(senstivity[n])
    
  return train_f_drug1,train_a_drug1,train_f_drug2,train_a_drug2,train_cell_line,test_f_drug1,test_a_drug1,test_f_drug2,test_a_drug2,test_cell_line,train_synergy,train_senstivity,test_synergy,test_senstivity


def preprocess(index_train,index_test):
    index_train1=[]
    index_test1=[]
    for i in range(len(index_train)):
        index_train1.append((index_train[i])[0])
        
    for ii in range(len(index_test)):
        index_test1.append((index_test[ii])[0])
        
    return index_train1,index_test1



def train_test_input2(f_drug1,a_drug1,f_drug2,a_drug2,cell_line,index_test,synergy,senstivity):
  
 
  train_a_drug1=a_drug1[3581:]
  train_a_drug2=a_drug2[3581:]
  train_f_drug1=f_drug1[3581:]
  train_f_drug2=f_drug2[3581:]
  train_cell_line=cell_line[3581:]
  train_synergy=synergy[3581:]
  train_senstivity=senstivity[3581:]
  test_a_drug1=a_drug1[0:3581]
  test_a_drug2=a_drug2[0:3581]
  test_f_drug1=f_drug1[0:3581]
  test_f_drug2=f_drug2[0:3581]
  test_cell_line=cell_line[0:3581]
  test_synergy=synergy[0:3581]
  test_senstivity=senstivity[0:3581]
    
#   train_synergy=np.array(train_synergy)
#   train_senstivity=np.array(train_senstivity)
#   test_synergy=np.array(test_synergy)
#   test_senstivity=np.array(test_senstivity)

  return train_f_drug1,train_a_drug1,train_f_drug2,train_a_drug2,train_cell_line,test_f_drug1,test_a_drug1,test_f_drug2,test_a_drug2,test_cell_line,train_synergy,train_senstivity,test_synergy,test_senstivity


def get_data_me1(s):
    

    !gdown --id 1lLKI0xPEbTBlkpu_6F3yoSYqf7pazg8d
    dele=pd.read_excel('deleted_index.xls',header=None)
    dele=np.array(dele)
    dele=dele-1
    
    
#     !gdown http://www.bioinf.jku.at/software/DeepSynergy/data_test_fold0_tanh.p.gz
#     file = gzip.open(data_file, 'rb')
#     X_tr, X_val, X_train, X_test, y_tr, y_val, y_train, y_test = pickle.load(file)
#     file.close()
    

    !gdown 1SvLGjU---dvwsuCTqRMehtV-Gmf2Hk52
    labels = pd.read_csv('pcbi.1006752.s004.csv', index_col=0) 
    
    h=len(np.array(labels))
    #labels are duplicated for the two different ways of ordering in the data
#     labels = pd.concat([labels, labels]) 
    
    test_fold =s
   
    idx_train = np.where(labels['Fold']!=test_fold)
    
#     h=len(idx_train[0])
    xx=np.where(idx_train == dele)[1]
    idx_train=np.delete(idx_train,xx)
    xx1=np.where(idx_train == (dele+h))[1]
    idx_train=np.delete(idx_train,xx1)
#     idx_train=idx_train[!dele]
    #indices of test data for model testing: fold 0
    idx_test = np.where(labels['Fold']==test_fold)
#     h1=len(idx_test[0])
    yy=np.where(idx_test == dele)[1]
    idx_test=np.delete(idx_test,yy)
    yy1=np.where(idx_test == (dele+h))[1]
    idx_test=np.delete(idx_test,yy1)
    
#     X_train = X[idx_train]
#     X_test = X[idx_test]
    idx_train1=idx_train + (idx_train.shape)
    idx_test1=idx_test + (idx_test.shape)
    
    idx_train1=idx_train + (h)
    idx_test1=idx_test + (h)
    idx_train1=np.r_[idx_train,idx_train1]
    idx_test1=np.r_[idx_test,idx_test1]
    #choose idx_train1 and idx_test1 if you want to duplicate data
    return idx_train1,idx_test1

def get_data_me(s):
   
    !gdown --id 1R_E1txnkHrwMQlBKpG7a4BHcZM4kRSkj
    dele=pd.read_excel('deleted_deep.xlsx',header=None)

    dele=np.array(dele)
    dele=dele-1
 
#     !gdown http://www.bioinf.jku.at/software/DeepSynergy/data_test_fold0_tanh.p.gz
#     file = gzip.open(data_file, 'rb')
#     X_tr, X_val, X_train, X_test, y_tr, y_val, y_train, y_test = pickle.load(file)
#     file.close()
    
    !gdown 1HNEch5czfqjThnZpFpP-Dcy0Qi-DQdc1
    labels = pd.read_csv('labels1.csv', index_col=0)


    h=len(np.array(labels))
    #labels are duplicated for the two different ways of ordering in the data
#     labels = pd.concat([labels, labels]) 
    
    test_fold =s
   
    idx_train = np.where(labels['fold']!=test_fold)
    
#     h=len(idx_train[0])
    xx=np.where(idx_train == dele)[1]
    idx_train=np.delete(idx_train,xx)
    xx1=np.where(idx_train == (dele+h))[1]
    idx_train=np.delete(idx_train,xx1)
#     idx_train=idx_train[!dele]
    #indices of test data for model testing: fold 0
    idx_test = np.where(labels['fold']==test_fold)
#     h1=len(idx_test[0])
    yy=np.where(idx_test == dele)[1]
    idx_test=np.delete(idx_test,yy)
    yy1=np.where(idx_test == (dele+h))[1]
    idx_test=np.delete(idx_test,yy1)
    
#     X_train = X[idx_train]
#     X_test = X[idx_test]

    y_train = labels.iloc[idx_train]['synergy'].values
    y_test = labels.iloc[idx_test]['synergy'].values
    y1_train = labels.iloc[idx_train]['senstivity'].values
    y1_test = labels.iloc[idx_test]['senstivity'].values
 
    return y_train,y_test,idx_train,idx_test,y1_train,y1_test



smiles,data_to_repeat,unique_drugs,unique_cell,deleted_index=get_data()
# smiles,data_to_repeat,unique_drugs,unique_cell,deleted_index=get_data()
deleted_index=deleted_index-1

feature,adjacency=graph_node(smiles)
# feature,edge_index=graph_node_edge(smiles)
data_to_repeat=np.r_[data_to_repeat,data_to_repeat]
l=int((data_to_repeat.shape[0])/2)
data_to_repeat[l:,0]=data_to_repeat[0:l,1]
data_to_repeat[l:,1]=data_to_repeat[0:l,0]
# np.random.shuffle(data_to_repeat)
synergy=data_to_repeat[:,13]
senstivity=data_to_repeat[:,5]
# synergy=data_to_repeat[:,3]
# senstivity=data_to_repeat[:,5]

# f_drug1,f_drug2,a_drug1,a_drug2,synergy,senstivity=repeat_smiles(data_to_repeat,unique_drugs,feature,adjacency,deleted_index,synergy,senstivity)

f_drug1,f_drug2,a_drug1,a_drug2,feature_cell=repeat_smiles1(data_to_repeat,unique_drugs,feature,adjacency,unique_cell)
# f_drug1,f_drug2,a_drug1,a_drug2,feature_cell=repeat_smiles_deep(data_to_repeat,unique_drugs,feature,adjacency,unique_cell)
# y_train,y_test,index_train,index_test,y1_train,y1_test=get_data_me(3)
index_train,index_test=get_data_me1(1)
# train_synergy1, train_senstivity1,test_synergy1, test_senstivity1,index_train,index_test=split_data(feature_cell,synergy,senstivity)
# feature_cell=np.array(feature_cell).astype(float)
# index_train,index_test=split_data1(feature_cell,synergy,senstivity)
# index_test,index_train=index1(1)
# index_train,index_test=preprocess(index_train,index_test)
# train_f_drug1,train_a_drug1,train_f_drug2,train_a_drug2,train_cell_line,test_f_drug1,test_a_drug1,test_f_drug2,test_a_drug2,test_cell_line,train_synergy, train_senstivity,test_synergy, test_senstivity=train_test_input1(f_drug1,a_drug1,f_drug2,a_drug2,feature_cell,index_test,synergy,senstivity)
train_f_drug1,train_a_drug1,train_f_drug2,train_a_drug2,train_cell_line,test_f_drug1,test_a_drug1,test_f_drug2,test_a_drug2,test_cell_line,train_synergy,train_senstivity,test_synergy,test_senstivity=train_test_input(f_drug1,a_drug1,f_drug2,a_drug2,feature_cell,index_train,index_test,synergy,senstivity)
train_cell_line=np.array(train_cell_line).astype(float)
test_cell_line=np.array(test_cell_line).astype(float)  
norm = "tanh_norm"
if norm == "tanh_norm":
    train_cell_line, mean, std, mean2, std2, feat_filt = normalize1(train_cell_line, norm=norm)
    test_cell_line, mean, std, mean2, std2, feat_filt = normalize1(test_cell_line, mean, std, mean2, std2, 
                                                          feat_filt=feat_filt, norm=norm)
else:
    train_cell_line, mean, std, feat_filt = normalize1(train_cell_line, norm=norm)
    test_cell_line, mean, std, feat_filt = normalize1(test_cell_line, mean, std, feat_filt=feat_filt, norm=norm)
  
print(synergy.shape)
# print(np.random.shuffle(dataset1))
# cc
# cc1=[m for m, v in enumerate(unique_name) if cc in v]
# # int((l))ik2
# unique_cell[0].shape[1]
# unique_name=unique_cell[:,0]
print(train_cell_line.shape)  
print(test_cell_line.shape)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.2/693.2 kB 2.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.1/29.1 MB 29.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for PubChemPy: filename=PubChemPy-1.0.4-py3-none-any.whl size=13834 sha256=860c4d21ad66fa907a717716de60dbbc9ccb9a454693011f0d357963a8a52063
  Stored in directory: /root/.cache/pip/wheels/7c/3d/8c/8192697412e9899dc55bbbb08bbc1197bef333caaa2a71c448
Successfully built PubChemPy
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.4/987.4 kB 3.7 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.3 MB/s eta 0:00:0000:01
  Created wheel for PyDrive: filename=PyDrive-1.3.1-py3-none-any.whl size=27453 sha256=e6d6b2b240533da29b31bfcd98e3cf9d075543c60a329888f94205de7f92ab2a
  Stored in directory: /root/.cache/pip/wheels/57/cc/07/6aac75f5395a224650905accd38c868c2276782a56f1046b7b
Successfully

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


hjjbjh
Downloading...
From: https://drive.google.com/uc?id=1m69fQ5l7q3dwQeud5M1nUVNhmrTGFbZc
To: /kaggle/working/pubchem.xls
100%|██████████████████████████████████████| 27.1k/27.1k [00:00<00:00, 35.1MB/s]
/opt/conda/lib/python3.7/site-packages/gdown/cli.py:125: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1z1sWG4E9BqipP0iczLrbR4qxSa4KG95_
To: /kaggle/working/pcbi.1006752.s004.xls
100%|███████████████████████████████████████| 5.24M/5.24M [00:00<00:00, 230MB/s]
Downloading...
From: https://drive.google.com/uc?id=1m69fQ5l7q3dwQeud5M1nUVNhmrTGFbZc
To: /kaggle/working/pubchem.xls
100%|██████████████████████████████████████| 27.1k/27.1k [00:00<00:00, 33.8MB/s]
/opt/conda/lib/python3.7/site-packages/gdown/cli.py:125: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need t

In [2]:
!pip install tensorflow
from networkx.readwrite.graph6 import data_to_n
from tensorflow.python.training.tracking import data_structures
!pip install spektral
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import add,PReLU, Flatten,Reshape,Dropout, Input,Dense,Add,concatenate,BatchNormalization, Activation#,MultiHeadAttention,AdditiveAttention
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
# from tensorflow.random import set_seed
from spektral.data.loaders import SingleLoader
from spektral.datasets.citation import Citation
from spektral.layers import GATConv, GlobalAvgPool,GlobalMaxPool, GCNConv,GlobalAttentionPool
from spektral.transforms import LayerPreprocess
# import tensorflow.compat.v1.keras.backend as K
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score#,AUC
from scipy.stats import pearsonr,spearmanr
from sklearn.metrics import roc_curve,auc
from keras.utils.vis_utils import plot_model
from tensorflow.keras import regularizers
def generate_network1(x_in1,a_in1,x_in2,a_in2,cell, inDrop, drop):
    # fill the architecture params from dict
    gcn_layers = [78,156,312]
    
    gan_layers = [128,128]
    cell_layers = [512,265,128]#for gcn
#     cell_layers = [2048,512,128*2] #for gan
    snp_layers = [512,128]#for gcn
#     snp_layers = [2048,512,256]# for gan
    dsn1_layers = [1024,2048,1024]
    dsn2_layers = [1024,2048,1024]
    N = x_in1[0].shape[0]
    F = x_in1[0].shape[1]
    
    channels = 8  # Number of channels in each head of the first GAT layer
    n_attn_heads = 10  # Number of attention heads in first GAT layer
    dropout = 0.2 # Dropout rate for the features and adjacency matrix
    drop=0.2
    l2_reg = 1e-3  # L2 regularization rate
    # contruct two parallel networks
    x_in1 = Input(shape=(N,F,),name='x_in1')
    a_in1 = Input((N,N,),name='a_in1')
    for l in range(len(gcn_layers)):
        if l == 0:
             
             do_1_d1 = Dropout(dropout)(x_in1)
             middle_layer_d1  = GCNConv(gcn_layers[l], activation='relu', use_bias=True, 
             kernel_initializer='glorot_uniform', 
             bias_initializer='zeros',kernel_regularizer=l2(l2_reg),
             bias_regularizer=l2(l2_reg), activty_regularizer=None,
             kernel_constraint=None, bias_constraint=None)([do_1_d1, a_in1])
#              middle_layer_d1=BatchNormalization()(middle_layer_d1)
        elif l == (len(gcn_layers)-1):
             middle_layer_d1 = GCNConv(gcn_layers[l], activation='relu', use_bias=True, 
             kernel_initializer='glorot_uniform', 
             bias_initializer='zeros',kernel_regularizer=l2(l2_reg),
             bias_regularizer=l2(l2_reg), activity_regularizer=None,
             kernel_constraint=None, bias_constraint=None)([middle_layer_d1, a_in1])
             gcn_output1=GlobalMaxPool(name='a1')(middle_layer_d1)
             
#              gcn_output1=Flatten()(x_in1)
#              gcn_output1=GlobalAttentionPool(1000)(x_in1)#(middle_layer_d1)
             
             
        else:
             middle_layer_d1 = Dropout(dropout)(middle_layer_d1)
             middle_layer_d1 = GCNConv(gcn_layers[l], activation='relu', use_bias=True, 
             kernel_initializer='glorot_uniform', 
             bias_initializer='zeros',kernel_regularizer=l2(l2_reg),
             bias_regularizer=l2(l2_reg), activity_regularizer=None,
             kernel_constraint=None, bias_constraint=None)([middle_layer_d1, a_in1])
#              middle_layer_d1=BatchNormalization()(middle_layer_d1)
#     print(x_in1.shape)
    # print(a_in1.shape)
#     for l in range(len(gan_layers)):
#         if l == 0:
#             middle_layer1  = GATConv(
#             gan_layers[l],
#             attn_heads=n_attn_heads,
#             concat_heads=True,
#             dropout_rate=dropout,
#             activation='elu',
#             kernel_regularizer=l2(l2_reg),
#             attn_kernel_regularizer=l2(l2_reg),
#             bias_regularizer=l2(l2_reg),
#              )([x_in1, a_in1])
            
#         elif l == (len(gan_layers)-1):
#             middle_layer1 = GATConv(
#             gan_layers[l],
#             dropout_rate=dropout,
#             activation='elu',
#             kernel_regularizer=l2(l2_reg),
#             attn_kernel_regularizer=l2(l2_reg),
#             bias_regularizer=l2(l2_reg),
#              )([middle_layer1, a_in1])
            
#             gan_output1=GlobalMaxPool()(middle_layer1)
# #             gan_output1 = Dense(int(gan_layers[l]), activation='tanh')(gan_output1)
#         else:
#             middle_layer1 = GATConv(
#             gan_layers[l],
#             dropout_rate=dropout,
#             activation='elu',
#             kernel_regularizer=l2(l2_reg),
#             attn_kernel_regularizer=l2(l2_reg),
#             bias_regularizer=l2(l2_reg),
#              )([middle_layer1, a_in1])
            
            
    
#   156  concatModel1 = concatenate([gcn_output1, gan_output1])
    gcn_output1 = Dense(256, activation='relu')(gcn_output1)
    gcn_output1 = Dropout(dropout)(gcn_output1)
    gcn_output1 = Dense(128, activation='linear')(gcn_output1)
    
    gan_output1 = Dense(128, activation='relu')(gan_output1)
   
    
    concatModel1 =  gan_output1
    # # addtModel = Add([gcn_output, gan_output])
    x_in2 = Input(shape=(N,F,),name='x_in2')
    a_in2 = Input((N,N,),name='a_in2')
    for l in range(len(gcn_layers)):
        if l == 0:
             
             do_1_d2 = Dropout(dropout)(x_in2)
             middle_layer_d2 =  GCNConv(gcn_layers[l], activation='relu', use_bias=True, 
             kernel_initializer='glorot_uniform', 
             bias_initializer='zeros',kernel_regularizer=l2(l2_reg),
             bias_regularizer=l2(l2_reg), activity_regularizer=None,
             kernel_constraint=None, bias_constraint=None)([do_1_d2, a_in2])
#              middle_layer_d2=BatchNormalization()(middle_layer_d2)
        elif l == (len(gcn_layers)-1):
             middle_layer_d2 = GCNConv(gcn_layers[l], activation='relu', use_bias=True, 
             kernel_initializer='glorot_uniform', 
             bias_initializer='zeros',kernel_regularizer=l2(l2_reg),
             bias_regularizer=l2(l2_reg), activity_regularizer=None,
             kernel_constraint=None, bias_constraint=None)([middle_layer_d2, a_in2])
             gcn_output2=GlobalMaxPool(name='a2')(middle_layer_d2)
#             gcn_output2=Flatten()(x_in2)
#             gcn_output2=GlobalAttentionPool(1000)(x_in2)#(middle_layer_d2)
             
        else:
             middle_layer_d2 = Dropout(dropout)(middle_layer_d2)
             middle_layer_d2 = GCNConv(gcn_layers[l], activation='relu', use_bias=True, 
             kernel_initializer='glorot_uniform', 
             bias_initializer='zeros',kernel_regularizer=l2(l2_reg),
             bias_regularizer=l2(l2_reg), activity_regularizer=None,
             kernel_constraint=None, bias_constraint=None)([middle_layer_d2, a_in2])
#              middle_layer_d2=BatchNormalization()(middle_layer_d2)

#     for l in range(len(gan_layers)):
#         if l == 0:
#             middle_layer2  = GATConv(
#             gan_layers[l],
#             attn_heads=n_attn_heads,
#             concat_heads=True,
#             dropout_rate=dropout,
#             activation='elu',
#             kernel_regularizer=l2(l2_reg),
#             attn_kernel_regularizer=l2(l2_reg),
#             bias_regularizer=l2(l2_reg),
#              )([x_in2, a_in2])
           
#         elif l == (len(gan_layers)-1):
#             middle_layer2 = GATConv(
#             gan_layers[l],
#             dropout_rate=dropout,
#             activation='elu',
#             kernel_regularizer=l2(l2_reg),
#             attn_kernel_regularizer=l2(l2_reg),
#             bias_regularizer=l2(l2_reg),
#              )([middle_layer2, a_in2])
#             gan_output2=GlobalMaxPool()(middle_layer2)
# #             gan_output2 = Dense(int(gan_layers[l]), activation='tanh')(gan_output2)
#         else:
#             middle_layer2 = GATConv(
#             gan_layers[l],
#             attn_heads=n_attn_heads,
#             concat_heads=True,
#             dropout_rate=dropout,
#             activation='elu',
#             kernel_regularizer=l2(l2_reg),
#             attn_kernel_regularizer=l2(l2_reg),
#             bias_regularizer=l2(l2_reg),
#              )([middle_layer2, a_in2])
    # print(gcn_output2.shape)
    # print(gan_output2.shape)        
#   156  concatModel2 = concatenate([gcn_output2, gan_output2])
    gcn_output2 = Dense(256, activation='relu')(gcn_output2)
    gcn_output2 = Dropout(dropout)(gcn_output2)
    gcn_output2 = Dense(128, activation='linear')(gcn_output2)
    
    gan_output2 = Dense(128, activation='relu')(gan_output2)
    
    concatModel2 = gcn_output2
    # concatModel2 = Add()([gcn_output2, gan_output2])
    for cell_layer in range(len(cell_layers)):
      if cell_layer == 0:
        input_cell    = Input(shape=(cell[0].shape[0],))
#         input_cell1 = BatchNormalization()(input_cell)
        cellFC = Dense(int(cell_layers[cell_layer]), activation='relu',use_bias=True,)(input_cell)
        cellFC = Dropout(float(drop))(cellFC)
#         cellFC=BatchNormalization()(cellFC)
      elif cell_layer == (len(cell_layers)-1):
        cellFC = Dense(int(cell_layers[cell_layer]), activation='relu',use_bias=True,)(cellFC)
#         cellFC=BatchNormalization()(cellFC)
      #  snp_output = Dense(1, activation='linear')(snpFC)
      else:
        cellFC = Dense(int(cell_layers[cell_layer]), activation='relu',use_bias=True,)(cellFC)
        cellFC = Dropout(float(drop))(cellFC)
#         cellFC=BatchNormalization()(cellFC)
    # print(concatModel1.shape)
    # print(cellFC.shape)
    
    concatModel=concatenate([concatModel2,concatModel1,cellFC])
    concatModel=BatchNormalization()(concatModel)
#     input2=concatenate([concatModel2,cellFC])
    # input1=concatenate([gan_output1,cellFC])
    # input2=concatenate([gan_output2,cellFC])
    # # addtModel = Add([concatModel1, concatModel2])
    # dsn1_layers = [2048,4096,2048]
    # dsn2_layers = [2048,4096,2048]
    # snp_layers = [2048,1024]
    # # contruct two parallel networks
    
       
#     for snp_layer in range(len(snp_layers)):
#        if snp_layer == 0:
#          snpFC1 = Dense(int(snp_layers[snp_layer]), activation='linear',use_bias=True,)(concatModel)
#          snpFC1=PReLU()(snpFC1)
#          snpFC = Dropout(float(drop))(snpFC1)
# #          snpFC=BatchNormalization()(snpFC)
#        elif snp_layer == (len(snp_layers)-1):
#          snpFC = Dense(int(snp_layers[snp_layer]), activation='linear',use_bias=True,)(snpFC)
#          snpFC=PReLU()(snpFC)
# #          layer = MultiHeadAttention(num_heads=2, key_dim=2)
# #          rr_task1=Reshape([1,snpFC.shape[1]])(snpFC)
# #          a_task1=layer(rr_task1,rr_task1)
#          snp_output1 = Dense(1, activation='linear')(snpFC)
#        else:
#           snpFC = Dense(int(snp_layers[snp_layer]), activation='linear',use_bias=True,)(snpFC)
#           snpFC=PReLU()(snpFC)
#           snpFC = Dropout(float(drop))(snpFC)


    for snp_layer in range(len(snp_layers)):
       if snp_layer == 0:
         snpFC1 = Dense(int(snp_layers[snp_layer]), activation='relu',use_bias=True,)(concatModel)
#          snpFC1=PReLU()(snpFC1)
         snpFC = Dropout(float(drop))(snpFC1)
#          snpFC=BatchNormalization()(snpFC)
       elif snp_layer == (len(snp_layers)-1):
         snpFC = Dense(int(snp_layers[snp_layer]), activation='relu',use_bias=True,)(snpFC)
#          snpFC=PReLU()(snpFC)
#          layer = MultiHeadAttention(num_heads=2, key_dim=2)
#          rr_task1=Reshape([1,snpFC.shape[1]])(snpFC)
#          a_task1=layer(rr_task1,rr_task1)
         snp_output1 = Dense(1, activation='linear')(snpFC)
       else:
          snpFC = Dense(int(snp_layers[snp_layer]), activation='relu',use_bias=True,)(snpFC)
#           snpFC=PReLU()(snpFC)
          snpFC = Dropout(float(drop))(snpFC)
#     model = Model(inputs=[x_in1,a_in1,x_in2,a_in2,input_cell],outputs= [f_task1,f_task2])
    model = Model(inputs=[x_in1,a_in1,x_in2,a_in2,input_cell],outputs= [snp_output1])
    # tf.keras.utils.plot_model(model.build_graph(), to_file="model.png",
    #        expand_nested=True, show_shapes=True)
    print(model.summary())
    return model

def trainer1(model, l_rate, train_f_drug1,train_a_drug1,train_f_drug2,train_a_drug2,train_cell_line,train_synergy,train_senstivity, epo, batch_size, earlyStop):
#     model.compile(loss='mse', optimizer=keras.optimizers.SGD(lr=float(l_rate), momentum=0.9,nesterov=True, clipvalue=0.3))
    model.compile(loss='mse', optimizer=keras.optimizers.Adam(lr=float(l_rate)))#, beta_1=0.9, beta_2=0.999, amsgrad=True))
    model.fit([train_f_drug1,train_a_drug1,train_f_drug2,train_a_drug2,train_cell_line], [train_synergy], shuffle=True, epochs=epo, batch_size=batch_size,verbose=1 )#,restore_best_weights=True,validation_split=0.1) 
                  #  validation_data=([val_input1,val_input2], [val_synergy,val_senstivity]))
    # model.fit({"input1":train_drug1, "input2":train_drug2}, {"synergy":train_synergy,"senstivity":train_senstivity}, epochs=epo, shuffle=True, batch_size=batch_size,verbose=1) 
    #                validation_data=({"input1":val_input1,"input2":val_input2}, {"synergy":val_synergy,"senstivity":val_senstivity}))

    return model
train_f_drug1=np.array(train_f_drug1)
train_a_drug1=np.array(train_a_drug1)
train_f_drug2=np.array(train_f_drug2)
train_a_drug2=np.array(train_a_drug2)
train_cell_line=np.array(train_cell_line)
test_f_drug1=np.array(test_f_drug1)
test_a_drug1=np.array(test_a_drug1)
test_f_drug2=np.array(test_f_drug2)
test_a_drug2=np.array(test_a_drug2)
test_cell_line=np.array(test_cell_line)
train_synergy=np.array(train_synergy)
train_senstivity=np.array(train_senstivity)
test_synergy=np.array(test_synergy)
test_senstivity=np.array(test_senstivity)


# train_synergy
# test_synergy
# train_synergy.shape
# test_synergy.shape
# train_senstivity

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 14.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 32.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 45.8 MB/s eta 0:00:0000:01:00:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.1.1
    Uninstalling typing_extensions-4.1.1:
      Successfully uninstalled typing_extensions-4.1.1
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: h5py
    Found existing installation: h5py 3.7.0
    Uninstalling h5py-3.7.0:
      Successfully uninstalled h5py-3.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.10.1
    Uninstalling tensorboard-2.10.1:
      Successfully uninstalled tensorboard-2.10.1
ERROR: pip's dependency resolver does not currently take into ac

In [3]:
### from sklearn.metrics import roc_curve,auc
l_rate = 0.0001
inDrop = 0.2
drop = 0.2
max_epoch = 500
# batch_size = 128 #gcn
batch_size = 64 #gan
earlyStop_patience = 20#np.ceil(train_f_drug1.shape[0]/batch_size)#1000

# model1= generate_network1(train_f_drug1,train_a_drug1,train_f_drug2,train_a_drug2,train_cell_line, inDrop, drop)
# plot_model(model1, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
# model1=trainer1(model1, l_rate, train_f_drug1,train_a_drug1,train_f_drug2,train_a_drug2,train_cell_line,train_synergy,train_senstivity,max_epoch, batch_size,
#                                 earlyStop_patience)
# # p1,p2= predict(model, [test_input1,test_input2])
# p1= model1.predict( [test_f_drug1,test_a_drug1,test_f_drug2,test_a_drug2,test_cell_line])
# # s1=np.zeros(len(p1))
# # s2=np.zeros(len(p1))
# # for i in range(len(p1)):
# #     n1=p1[0]
# #     h1=n1[0]
# #     x1=h1[0]
# #     s1[i]=x1
# #     n2=p2[i]
# #     h2=n2[0]
# #     x2=h2[0]

# synergy_error=mean_squared_error(test_synergy, p1)
# # senstivity_error=mean_squared_error(test_senstivity, p2)
# # p1,p2

# # senstivity_error=mean_squared_error(test_senstivity, p2)
# synergy_error1=mean_absolute_error(test_synergy, p1)
# # senstivity_error1=mean_absolute_error(test_senstivity, p2)
# p12=[]
# for i in range(len(p1)):
#   x=p1[i]
#   p12.append(x[0])
# p12=np.array(p12)
# synergy_error2=r2_score(test_synergy, p12)
# # senstivity_error2=r2_score(test_senstivity, p2)
# print("synergy_mean_squared_error",synergy_error)
# # print("senstivity_mean_squared_error",senstivity_error)
# print("synergy_mean_absolute_error",synergy_error1)
# # print("senstivity_mean_absolute_error",senstivity_error1)
# print("synergy_r2_score",synergy_error2)
# # print("senstivity_r2_score",senstivity_error2)
# losses = model1.evaluate([test_f_drug1,test_a_drug1,test_f_drug2,test_a_drug2,test_cell_line],[test_synergy],batch_size =64)
# print(losses)


# synergy_pear= pearsonr(test_synergy, p12)
# synergy_spear= spearmanr(test_synergy, p12)
# print("synergy_pear",synergy_pear)
# print("synergy_spear",synergy_spear)
# p1
# # senstivity_pear= pearsonr(test_senstivity, p2)
# # senstivity_spear= spearmanr(test_senstivity, p2)
# # print("senstivity_pear",senstivity_pear)
# # print("senstivity_spear",senstivity_spear)


In [4]:

import numpy as np
import matplotlib.pyplot as plt

# import tensorflow_datasets as tfds
import tensorflow as tf
def scaled_dot_product_attention(q, k, v, mask=None):
  """Calculate the attention weights.
  q, k, v must have matching leading dimensions.
  k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
  The mask has different shapes depending on its type(padding or look ahead)
  but it must be broadcastable for addition.

  Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable
          to (..., seq_len_q, seq_len_k). Defaults to None.

  Returns:
    output, attention_weights
  """

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

  # scale matmul_qk
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  # add the mask to the scaled tensor.
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)

  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  attention_weights = tf.nn.softmax(scaled_attention_logits)#, axis=-1)#tf.nn.linear()#, axis=-1)  # (..., seq_len_q, seq_len_k)

  output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

  return output,attention_weights 

#multi-head

class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self,*, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.wq = tf.keras.layers.Dense(d_model,activation='relu',use_bias='true')
    self.wk = tf.keras.layers.Dense(d_model,activation='relu',use_bias='true')
    self.wv = tf.keras.layers.Dense(d_model,activation='relu',use_bias='true')

    self.dense = tf.keras.layers.Dense(d_model)#,activation='relu',use_bias='true')

  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    """
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])

  def call(self, v, k, q, mask=None):
    batch_size = tf.shape(q)[0]

    q = self.wq(q)  # (batch_size, seq_len, d_model)
#     q=PReLU()(q)
    k = self.wk(k)  # (batch_size, seq_len, d_model)
#     k=PReLU()(k)
    v = self.wv(v)  # (batch_size, seq_len, d_model)
#     v=PReLU()(v)

    q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
    k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

    return output#, attention_weights



class CrossStitch(tf.keras.layers.Layer):

    """Cross-Stitch implementation according to arXiv:1604.03539
    Implementation adapted from https://github.com/helloyide/Cross-stitch-Networks-for-Multi-task-Learning"""

    def __init__(self, num_tasks, *args, **kwargs):
        """initialize class variables"""
        self.num_tasks = num_tasks
        super(CrossStitch, self).__init__(**kwargs)

    def build(self, input_shape):
        """initialize the kernel and set the instance to 'built'"""
        self.kernel = self.add_weight(name="kernel",
                                      shape=(self.num_tasks,
                                             self.num_tasks),
                                      initializer='identity',
                                      trainable=True)
        super(CrossStitch, self).build(input_shape)

    def call(self, xl):
        """
        called by TensorFlow when the model gets build. 
        Returns a stacked tensor with num_tasks channels in the 0 dimension, 
        which need to be unstacked.
        """
        if (len(xl) != self.num_tasks):
            # should not happen
            raise ValueError()

        out_values = []
        for this_task in range(self.num_tasks):
            this_weight = self.kernel[this_task, this_task]
            out = tf.math.scalar_mul(this_weight, xl[this_task])
            for other_task in range(self.num_tasks):
                if this_task == other_task:
                    continue  # already weighted!
                other_weight = self.kernel[this_task, other_task]
                out += tf.math.scalar_mul(other_weight, xl[other_task])
            out_values.append(out)
        # HACK!
        # unless we stack, and then unstack the tensors, TF (2.0.0) can't follow
        # the graph, so it aborts during model initialization.
        # return tf.stack(out_values, axis=0)
        return out_values[0],out_values[1]

    def compute_output_shape(self, input_shape):
        return [self.num_tasks] + input_shape

    def get_config(self):
        """implemented so keras can save the model to json/yml"""
        config = {
            "num_tasks": self.num_tasks
        }
        base_config = super(CrossStitch, self).get_config()
        return dict(list(config.items()) + list(base_config.items()))

    


In [5]:

def generate_network_att1(x_in1,a_in1,x_in2,a_in2,cell, inDrop, drop):
    # fill the architecture params from dict
#     gcn_layers = [1024,512,156]
#     gan_layers = [1024,512]
#     cell_layers = [2048,512]#,2048]
#     snp_layers = [1024,512,128]#,2048]
#     dsn1_layers = [1024,2048,1024]
#     dsn2_layers = [1024,2048,1024]
    # fill the architecture params from dict
    gcn_layers = [78,156,312]
#     gan_layers = [128,128]
#     gcn_layers = [32,64,128]
    cell_layers = [512,265,128]#for gcn
#     cell_layers = [2048,512,256] #for gan
    snp_layers = [512,128]#for gcn
#     snp_layers = [2048,512,256]# for gan
    dsn1_layers = [1024,2048,1024]
    dsn2_layers = [1024,2048,1024]
    N = x_in1[0].shape[0]
    F = x_in1[0].shape[1]
    
    channels = 8  # Number of channels in each head of the first GAT layer
    n_attn_heads = 10  # Number of attention heads in first GAT layer
    dropout = 0.2 # Dropout rate for the features and adjacency matrix
    drop=0.2
    l2_reg = 1e-3
    N = x_in1[0].shape[0]
    F = x_in1[0].shape[1]
    
    channels = 8  # Number of channels in each head of the first GAT layer
    n_attn_heads = 8  # Number of attention heads in first GAT layer
    dropout = 0.2 # Dropout rate for the features and adjacency matrix
    drop=0.2
    l2_reg = 1e-3  # L2 regularization rate
    # contruct two parallel networks
    x_in1 = Input(shape=(N,F,),name='x_in1')
    a_in1 = Input((N,N,),name='a_in1')
    for l in range(len(gcn_layers)):
        if l == 0:
             
             do_1_d1 = Dropout(dropout)(x_in1)
             middle_layer_d1  = GCNConv(gcn_layers[l], activation='relu', use_bias=True, 
             kernel_initializer='glorot_uniform', 
             bias_initializer='zeros',kernel_regularizer=l2(l2_reg),
             bias_regularizer=l2(l2_reg), activty_regularizer=None,
             kernel_constraint=None, bias_constraint=None)([do_1_d1, a_in1])
#              middle_layer_d1=concatenate([middle_layer_d1,do_1_d1])
        elif l == (len(gcn_layers)-1):
             middle_layer_d1 = GCNConv(gcn_layers[l], activation='relu', use_bias=True, 
             kernel_initializer='glorot_uniform', 
             bias_initializer='zeros',kernel_regularizer=l2(l2_reg),
             bias_regularizer=l2(l2_reg), activity_regularizer=None,
             kernel_constraint=None, bias_constraint=None)([middle_layer_d1, a_in1])
             gcn_output1=GlobalMaxPool()(middle_layer_d1)
#              gcn_output1=GlobalAttentionPool(gcn_layers[l],name='a1')(middle_layer_d1)
             
#              gcn_output1=Flatten()(x_in1)
#              gcn_output1=GlobalAttentionPool(1000)(x_in1)#(middle_layer_d1)
             
             
        else:
             middle_layer_d1 = Dropout(dropout)(middle_layer_d1)
             middle_layer_d1 = GCNConv(gcn_layers[l], activation='relu', use_bias=True, 
             kernel_initializer='glorot_uniform', 
             bias_initializer='zeros',kernel_regularizer=l2(l2_reg),
             bias_regularizer=l2(l2_reg), activity_regularizer=None,
             kernel_constraint=None, bias_constraint=None)([middle_layer_d1, a_in1])
#              middle_layer_d1=concatenate([middle_layer_d1,middle_layer_d11])
#              middle_layer_d1=BatchNormalization()(middle_layer_d1)
#     print(x_in1.shape)
    # print(a_in1.shape)
#     for l in range(len(gan_layers)):
#         if l == 0:
#             middle_layer1  = GATConv(
#             gan_layers[l],
#             attn_heads=n_attn_heads,
#             concat_heads=True,
#             dropout_rate=dropout,
#             activation='elu',
#             kernel_regularizer=l2(l2_reg),
#             attn_kernel_regularizer=l2(l2_reg),
#             bias_regularizer=l2(l2_reg),
#              )([x_in1, a_in1])
#             middle_layer1=concatenate([middle_layer1,x_in1]) 
#         elif l == (len(gan_layers)-1):
#             middle_layer1 = GATConv(
#             gan_layers[l],
#             dropout_rate=dropout,
#             activation='elu',
#             kernel_regularizer=l2(l2_reg),
#             attn_kernel_regularizer=l2(l2_reg),
#             bias_regularizer=l2(l2_reg),
#              )([middle_layer1, a_in1])
#             gan_output1=GlobalAttentionPool(512)(middle_layer1)
# #             gan_output1 = Dense(int(gan_layers[l]), activation='elu')(gan_output1)
#         else:
#             middle_layer11 = GATConv(
#             gan_layers[l],
#             attn_heads=n_attn_heads,
#             concat_heads=True,
#             dropout_rate=dropout,
#             activation='elu',
#             kernel_regularizer=l2(l2_reg),
#             attn_kernel_regularizer=l2(l2_reg),
#             bias_regularizer=l2(l2_reg),
#              )([middle_layer1, a_in1])
#             middle_layer1=concatenate([middle_layer1,middle_layer11])
    
#  156   concatModel1 = concatenate([gcn_output1, gan_output1])
    gcn_output1 = Dense(256, activation='relu',kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(gcn_output1)
    gcn_output1 = Dropout(dropout)(gcn_output1)
    gcn_output1 = Dense(128, activation='linear',kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(gcn_output1)
    
#     gan_output1 = Dense(128, activation='relu')(gan_output1)
    
    concatModel1 =  gcn_output1
    # # addtModel = Add([gcn_output, gan_output])
    x_in2 = Input(shape=(N,F,),name='x_in2')
    a_in2 = Input((N,N,),name='a_in2')
    for l in range(len(gcn_layers)):
        if l == 0:
             
             do_1_d2 = Dropout(dropout)(x_in2)
             middle_layer_d2 =  GCNConv(gcn_layers[l], activation='relu', use_bias=True, 
             kernel_initializer='glorot_uniform', 
             bias_initializer='zeros',kernel_regularizer=l2(l2_reg),
             bias_regularizer=l2(l2_reg), activity_regularizer=None,
             kernel_constraint=None, bias_constraint=None)([do_1_d2, a_in2])
#              middle_layer_d2=concatenate([middle_layer_d2,do_1_d2])
#              middle_layer_d2=BatchNormalization()(middle_layer_d2)
        elif l == (len(gcn_layers)-1):
             middle_layer_d2 = GCNConv(gcn_layers[l], activation='relu', use_bias=True, 
             kernel_initializer='glorot_uniform', 
             bias_initializer='zeros',kernel_regularizer=l2(l2_reg),
             bias_regularizer=l2(l2_reg), activity_regularizer=None,
             kernel_constraint=None, bias_constraint=None)([middle_layer_d2, a_in2])
#              gcn_output2=GlobalAttentionPool(gcn_layers[l],name='a2')(middle_layer_d2)
             gcn_output2=GlobalMaxPool()(middle_layer_d2)
#             gcn_output2=Flatten()(x_in2)
#             gcn_output2=GlobalAttentionPool(1000)(x_in2)#(middle_layer_d2)
             
        else:
             middle_layer_d2 = Dropout(dropout)(middle_layer_d2)
             middle_layer_d2 = GCNConv(gcn_layers[l], activation='relu', use_bias=True, 
             kernel_initializer='glorot_uniform', 
             bias_initializer='zeros',kernel_regularizer=l2(l2_reg),
             bias_regularizer=l2(l2_reg), activity_regularizer=None,
             kernel_constraint=None, bias_constraint=None)([middle_layer_d2, a_in2])
#              middle_layer_d2=concatenate([middle_layer_d2,middle_layer_d21])
#              middle_layer_d2=BatchNormalization()(middle_layer_d2)

#     for l in range(len(gan_layers)):
#         if l == 0:
            
#             middle_layer2  = GATConv(
#             gan_layers[l],
#             attn_heads=n_attn_heads,
#             concat_heads=True,
#             dropout_rate=dropout,
#             activation='elu',
#             kernel_regularizer=l2(l2_reg),
#             attn_kernel_regularizer=l2(l2_reg),
#             bias_regularizer=l2(l2_reg),
#              )([x_in2, a_in2])
#             middle_layer2=concatenate([middle_layer2,x_in2])
#         elif l == (len(gan_layers)-1):
#             middle_layer2 = GATConv(
#             gan_layers[l],
#             attn_heads=n_attn_heads,
#             concat_heads=True,
#             dropout_rate=dropout,
#             activation='elu',
#             kernel_regularizer=l2(l2_reg),
#             attn_kernel_regularizer=l2(l2_reg),
#             bias_regularizer=l2(l2_reg),
#              )([middle_layer2, a_in2])
#             gan_output2=GlobalAttentionPool(512)(middle_layer2)
# #             gan_output2 = Dense(int(gan_layers[l]), activation='tanh')(gan_output2)
#         else:
#             middle_layer21 = GATConv(
#             gan_layers[l],
#             attn_heads=n_attn_heads,
#             concat_heads=True,
#             dropout_rate=dropout,
#             activation='elu',
#             kernel_regularizer=l2(l2_reg),
#             attn_kernel_regularizer=l2(l2_reg),
#             bias_regularizer=l2(l2_reg),
#              )([middle_layer2, a_in2])
#             middle_layer2=concatenate([middle_layer2,middle_layer_d21])
    # print(gcn_output2.shape)
    # print(gan_output2.shape)        
#  156   concatModel2 = concatenate([gcn_output2, gan_output2])
    gcn_output2 = Dense(256, activation='relu',kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(gcn_output2)
    gcn_output2 = Dropout(dropout)(gcn_output2)
    gcn_output2 = Dense(128, activation='linear',kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(gcn_output2)
    
#     gan_output2 = Dense(128, activation='relu')(gan_output2)
        
    concatModel2 = gcn_output2
    # concatModel2 = Add()([gcn_output2, gan_output2])
    for cell_layer in range(len(cell_layers)):
      if cell_layer == 0:
        input_cell    = Input(shape=(cell[0].shape[0],))
#         input_cell1 = BatchNormalization()(input_cell)
        cellFC = Dense(int(cell_layers[cell_layer]), activation='relu',use_bias=True,kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(input_cell)
        cellFC = Dropout(float(drop))(cellFC)
#         cellFC=BatchNormalization()(cellFC)
      elif cell_layer == (len(cell_layers)-1):
        cellFC = Dense(int(cell_layers[cell_layer]), activation='linear',use_bias=True,kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(cellFC)
#         cellFC=BatchNormalization()(cellFC)
      #  snp_output = Dense(1, activation='linear')(snpFC)
      else:
        cellFC = Dense(int(cell_layers[cell_layer]), activation='relu',use_bias=True,kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(cellFC)
        cellFC = Dropout(float(drop))(cellFC)
#         cellFC=BatchNormalization()(cellFC)
    # print(concatModel1.shape)
    # print(cellFC.shape)
    
    concatModel=concatenate([concatModel2,concatModel1,cellFC])
    concatModel=BatchNormalization()(concatModel)
#     input2=concatenate([concatModel2,cellFC])
    # input1=concatenate([gan_output1,cellFC])
    # input2=concatenate([gan_output2,cellFC])
    # # addtModel = Add([concatModel1, concatModel2])
    # dsn1_layers = [2048,4096,2048]
    # dsn2_layers = [2048,4096,2048]
    # snp_layers = [2048,1024]
    # # contruct two parallel networks
  
    
#     task1=Dense(512, activation='relu',use_bias=True)(concatModel)
# #     task1=PReLU()(task1)
#     task2=Dense(512, activation='relu',use_bias=True)(concatModel)
    
    layer1 =MultiHeadAttention(d_model=concatModel.shape[1], num_heads=4)
    a_task1= layer1(concatModel,concatModel,concatModel)
    layer2 = MultiHeadAttention(d_model=concatModel.shape[1], num_heads=4)
    a_task2= layer2(concatModel,concatModel,concatModel)
    task11 = Reshape([a_task1.shape[2]])(a_task1)
    task22 = Reshape([a_task2.shape[2]])(a_task2)
    task1=concatenate([task11,concatModel])
    task2=concatenate([task22,concatModel])
#     task2=PReLU()(task2)
#     task1=BatchNormalization()(task1)
#     task2=BatchNormalization()(task2)
    r_task1,r_task2 = CrossStitch(2)([task1,task2])
#     r_task1,r_task2 = CrossStitch(2)([r_task1,r_task2])
#     r_task1,r_task2 = CrossStitch(2)([r_task1,r_task2])
#     r_task1,r_task2 = apply_cross_stitch([task1,task2])
#     r_task1=PReLU()(r_task1)
#     r_task2=PReLU()(r_task2)

#     r_task1=concatenate([r_task1,task1])
#     r_task2=concatenate([r_task2,task2])
#     r_task1=BatchNormalization()(r_task1)
#     r_task2=BatchNormalization()(r_task2)
#     r_task1=concatenate([r_task1,task1])
#     r_task2=concatenate([r_task2,task2])
    
#     r_task1=Dense(256, activation='relu',use_bias=True)(r_task1)
    
#     r_task2=Dense(256, activation='relu',use_bias=True)(r_task2)
#     r_task1,r_task2 = CrossStitch(2)([r_task1,r_task2])
    
    
    r_task1=Dense(1024,activation='relu',use_bias=True,kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(r_task1)
    
    r_task2=Dense(1024, activation='relu',use_bias=True,kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(r_task2)
    r_task1,r_task2 = CrossStitch(2)([r_task1,r_task2])
    
#     layer1 =MultiHeadAttention(d_model=r_task1.shape[1], num_heads=4)
#     a_task1= layer1(r_task1,r_task1,r_task1)
#     layer2 = MultiHeadAttention(d_model=r_task2.shape[1], num_heads=4)
#     a_task2= layer2(r_task2,r_task2,r_task2)
#     task11 = Reshape([a_task1.shape[2]])(a_task1)
#     task22 = Reshape([a_task2.shape[2]])(a_task2)
#     r_task1=concatenate([task11,r_task1])
#     r_task2=concatenate([task22,r_task2])
    
    r_task1=concatenate([r_task1,task1])
    r_task2=concatenate([r_task2,task2])
    
#     r_task1=PReLU()(r_task1)
#     r_task2=PReLU()(r_task2)

 
   
#     r_task1,r_task2 = CrossStitch(2)([r_task1,r_task2])
#     r_task1,r_task2 = apply_cross_stitch([r_task1,r_task2])

    
#     r_task1=BatchNormalization()(r_task1)
#     r_task2=BatchNormalization()(r_task2)
    
#     r_task1=Dense(128, activation='linear',use_bias=True)(r_task1)
    
#     r_task2=Dense(128, activation='linear',use_bias=True)(r_task2)
#     r_task1=PReLU()(r_task1)
#     r_task2=PReLU()(r_task2)
#     r_task1,r_task2 = CrossStitch(2)([r_task1,r_task2])
#     r_task1,r_task2 = apply_cross_stitch(r_task1,r_task2)

    
#     r_task1=BatchNormalization()(r_task1)
#     r_task2=BatchNormalization()(r_task2)
    
     
#     layer1 =MultiHeadAttention(d_model=r_task1.shape[1], num_heads=4)
#     a_task1= layer1(r_task1,r_task1,r_task1)
#     layer2 = MultiHeadAttention(d_model=r_task2.shape[1], num_heads=4)
#     a_task2= layer2(r_task2,r_task2,r_task2)
#     a_task1 = Reshape([a_task1.shape[2]])(a_task1)
#     a_task1=BatchNormalization()(a_task1)
#     a_task2 = Reshape([a_task2.shape[2]])(a_task2)
#     a_task2=BatchNormalization()(a_task2)
    
    
#     r_task1,r_task2 = CrossStitch(2)([r_task1,r_task2])
#     r_task1,r_task2 = CrossStitch(2)([r_task1,r_task2])
#     r_task1,r_task2 = CrossStitch(2)([r_task1,r_task2])
# #  
#     rr_task2=Reshape([1,r_task2.shape[1]])(r_task2)
#     a_task1=layer(rr_task1,rr_task1)
#     a_task2=layer(rr_task2,rr_task2)
#     a_task1 = Reshape([a_task1.shape[2]])(a_task1)
#     a_task2 = Reshape([a_task2.shape[2]])(a_task2)
    
#     layer1 = MultiHeadAttention(d_model=r_task1.shape[1], num_heads=8)
#     a_task1= layer1(r_task1,r_task1,r_task1)
#     layer2 = MultiHeadAttention(d_model=r_task2.shape[1], num_heads=8)
#     a_task2= layer2(r_task2,r_task2,r_task2)
#     a_task1 = Reshape([a_task1.shape[2]])(a_task1)
#     a_task2 = Reshape([a_task2.shape[2]])(a_task2)
    
#     r_task1 = Dense(64, activation='linear',name="synergy")(a_task1)
#     r_task2 = Dense(64, activation='linear',name="senstivity")(a_task2)
    
#     layer1 = MultiHeadAttention(d_model=r_task1.shape[1], num_heads=8)
#     a_task1= layer1(r_task1,r_task1,r_task1)
#     layer2 = MultiHeadAttention(d_model=r_task2.shape[1], num_heads=8)
#     a_task2= layer2(r_task2,r_task2,r_task2)
#     a_task1 = Reshape([a_task1.shape[2]])(a_task1)
#     a_task1=BatchNormalization()(a_task1)
#     a_task2 = Reshape([a_task2.shape[2]])(a_task2)
#     a_task2=BatchNormalization()(a_task2)
   
    r_task1 = Dense(128, activation='linear',kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(r_task1)
    r_task1=PReLU()(r_task1)
    r_task2 = Dense(128, activation='relu',kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(r_task2)
    r_task2=PReLU()(r_task2)
    r_task1 = Dense(64, activation='linear',name="synergy",kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(r_task1)
    r_task1=PReLU()(r_task1)
    r_task2 = Dense(64, activation='relu',name="senstivity",kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(r_task2)
# 
#     r_task1 = Dense(512, activation='relu')(r_task1)
# #   
#     r_task2 = Dense(512, activation='relu')(r_task2)

#     r_task1 = Dense(256, activation='relu',name="synergy")(r_task1)
#     r_task2 = Dense(256, activation='relu',name="senstivity")(r_task2)
    
    snp_output1 = Dense(1, activation='linear')(r_task1)
#     snp_output1=PReLU()(snp_output1)
    snp_output2 = Dense(1, activation='relu')(r_task2)
#     model = Model(inputs=[x_in1,a_in1,x_in2,a_in2,input_cell],outputs= [f_task1,f_task2])
    model = Model(inputs=[x_in1,a_in1,x_in2,a_in2,input_cell],outputs= [snp_output1,snp_output2])
    # tf.keras.utils.plot_model(model.build_graph(), to_file="model.png",
    #        expand_nested=True, show_shapes=True)
    print(model.summary())
    return model

def trainer_att1(model, l_rate, train_f_drug1,train_a_drug1,train_f_drug2,train_a_drug2,train_cell_line,train_synergy,train_senstivity, epo, batch_size, earlyStop):
    model.compile(loss='mse', optimizer=keras.optimizers.Adam(lr=float(l_rate)))#, beta_1=0.9, beta_2=0.999, amsgrad=True))
#     model.compile(loss='mse', optimizer=keras.optimizers.SGD(lr=float(l_rate), momentum=0.9, nesterov=True, clipnorm=1.))
    model.fit([train_f_drug1,train_a_drug1,train_f_drug2,train_a_drug2,train_cell_line], [train_synergy,train_senstivity], shuffle=True, epochs=epo, batch_size=batch_size,verbose=1 )#,restore_best_weights=True,validation_split=0.1) 
                  #  validation_data=([val_input1,val_input2], [val_synergy,val_senstivity]))
    # model.fit({"input1":train_drug1, "input2":train_drug2}, {"synergy":train_synergy,"senstivity":train_senstivity}, epochs=epo, shuffle=True, batch_size=batch_size,verbose=1) 
    #                validation_data=({"input1":val_input1,"input2":val_input2}, {"synergy":val_synergy,"senstivity":val_senstivity}))

    return model

In [6]:
# l_rate = 0.0001
# inDrop = 0.2
# drop = 0.2
# max_epoch = 500
# batch_size =16


# earlyStop_patience = np.ceil(train_f_drug1.shape[0]/batch_size)#1000
model_att1= generate_network_att1(train_f_drug1,train_a_drug1,train_f_drug2,train_a_drug2,train_cell_line, inDrop, drop)
model_att1=trainer_att1(model_att1, l_rate, train_f_drug1,train_a_drug1,train_f_drug2,train_a_drug2,train_cell_line,train_synergy,train_senstivity,max_epoch, batch_size,
                                earlyStop_patience)

# p1,p2= predict(model, [test_input1,test_input2])
ap111,ap221= model_att1.predict( [test_f_drug1,test_a_drug1,test_f_drug2,test_a_drug2,test_cell_line])
asynergy_error1=mean_squared_error(test_synergy, ap111)
asenstivity_error1=mean_squared_error(test_senstivity, ap221)
asynergy_error11=mean_absolute_error(test_synergy, ap111)
asenstivity_error11=mean_absolute_error(test_senstivity, ap221)
asynergy_error21=r2_score(test_synergy, ap111)
asenstivity_error21=r2_score(test_senstivity, ap221)
print("msynergy_mean_squared_error",asynergy_error1)
print("msenstivity_mean_squared_error",asenstivity_error1)
print("msynergy_mean_absolute_error",asynergy_error11)
print("msenstivity_mean_absolute_error",asenstivity_error11)
print("msynergy_r2_score",asynergy_error21)
print("msenstivity_r2_score",asenstivity_error21)
cross_att1 = model_att1.evaluate([test_f_drug1,test_a_drug1,test_f_drug2,test_a_drug2,test_cell_line],[test_synergy,test_senstivity],batch_size =64)
print(cross_att1)
asynergy_pear1= pearsonr(test_synergy, ap111)
asynergy_spear1= spearmanr(test_synergy, ap111)
print("msynergy_pear",asynergy_pear1)
print("msynergy_spear",asynergy_spear1)
asenstivity_pear1= pearsonr(test_senstivity, ap221)
asenstivity_spear1= spearmanr(test_senstivity, ap221)
print("msenstivity_pear",asenstivity_pear1)
print("msenstivity_spear",asenstivity_spear1)
ap111,ap221
# yourTerminal:prompt> jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10 


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x_in2 (InputLayer)              [(None, 69, 32)]     0                                            
__________________________________________________________________________________________________
x_in1 (InputLayer)              [(None, 69, 32)]     0                                            
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 69, 32)       0           x_in2[0][0]                      
__________________________________________________________________________________________________
a_in2 (InputLayer)              [(None, 69, 69)]     0                                            
______________________________________________________________________________________________

/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
2022-12-12 14:45:09.896377: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 254820864 exceeds 10% of free system memory.
2022-12-12 14:45:10.375408: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 549457488 exceeds 10% of free system memory.
2022-12-12 14:45:11.058207: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 254820864 exceeds 10% of free system memory.
2022-12-12 14:45:11.612306: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 549457488 exceeds 10% of free system memory.
2022-12-12 14:45:12.450550: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 254820864 exceeds 10% of free system memory.


Epoch 1/500
451/451 [==============================] - 12s 15ms/step - loss: 778.7999 - dense_19_loss: 517.5131 - dense_20_loss: 132.2431
Epoch 2/500
451/451 [==============================] - 7s 16ms/step - loss: 700.0775 - dense_19_loss: 482.5902 - dense_20_loss: 91.9822
Epoch 3/500
451/451 [==============================] - 7s 16ms/step - loss: 675.2711 - dense_19_loss: 466.8318 - dense_20_loss: 84.9942
Epoch 4/500
451/451 [==============================] - 7s 15ms/step - loss: 655.4475 - dense_19_loss: 453.9673 - dense_20_loss: 79.6841
Epoch 5/500
451/451 [==============================] - 7s 15ms/step - loss: 635.5797 - dense_19_loss: 439.9186 - dense_20_loss: 75.1273
Epoch 6/500
451/451 [==============================] - 7s 16ms/step - loss: 618.0560 - dense_19_loss: 426.1363 - dense_20_loss: 72.5509
Epoch 7/500
451/451 [==============================] - 7s 15ms/step - loss: 601.2322 - dense_19_loss: 413.4291 - dense_20_loss: 69.4450
Epoch 8/500
451/451 [=========================

(array([[-8.784463 ],
        [ 4.5820017],
        [ 2.0213995],
        ...,
        [ 6.754138 ],
        [ 4.641059 ],
        [ 8.13182  ]], dtype=float32),
 array([[30.180082 ],
        [ 2.1186607],
        [17.688425 ],
        ...,
        [16.93225  ],
        [38.494675 ],
        [23.287607 ]], dtype=float32))

In [11]:

from IPython.display import FileLink
np.savetxt('pred_syn1.csv', ap111 ,delimiter=',')
FileLink(r'pred_syn1.csv')

np.savetxt('pred_sen1.csv', ap221 ,delimiter=',')
FileLink(r'pred_sen1.csv')

np.savetxt('test_syn1.csv', test_synergy ,delimiter=',')
FileLink(r'test_syn1.csv')

np.savetxt('test_sen1.csv', test_senstivity ,delimiter=',')
FileLink(r'test_sen1.csv')

/kaggle/working/test_sen1.csv